## Read in data
In this tutorial, it required three input data:

1. "resnet_array.npy": a matrix of image features for each spot that extract from ResNet50
2. "filtered_feature_bc_matrix.h5": The gene expression matrix
3. "tissue_positions_list.csv": Spatial coordinateds 
4. "image" folder: contains cropped image for each spot

The gene expreesion data can be stored as an AnnData object. AnnData stores a data matrix .X together with annotations of observations .obs and variables .var 

The files "resnet_array.npy", "tissue_positions_list.csv" will be utilized in the adjacency matrix module, while the file "filtered_feature_bc_matrix.h5" will be used in the feature matrix module.

In [1]:
import pandas as pd
import numpy as np
import os
import re
import sys
import fileinput


path = "/Users/ninasong/Desktop/spatialProject/literature_model/graph_convolutional_clustering/unsupervised-GCN/FFD1"

array = np.load((os.path.join(path, "resnet_array.npy")))
len(array)
# resnet_feature_matrix = pd.read_csv('spatialDataset/10xDemo/mouse_brain/annotation/top10_resetnet_feature.csv', sep=',')

3594

# Adjcency matrix

To prepare the adjacency matrix, the features for each spot will consist of the top 2 principal components from Resnet analysis, which have been subject to standard scale normalization. Additionally, two position features will be included for each spot, which will be normalized using min-max normalization. The cosine distance will be calculated between spots, and if the distance is negative, it will be set to zero.

"resnet_array.npy" contains 2048 features per spot, and principal component analysis will be conducted to extract the top two PCs.

In [2]:
from sklearn.preprocessing import StandardScaler

# assume X is a matrix of size (n_samples, n_features)
scaler = StandardScaler()
array_normalized = scaler.fit_transform(array)

In [3]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
resne_fea = pca.fit_transform(array_normalized)
top2_resnet = pd.DataFrame(resne_fea)

top2_resnet.to_csv((os.path.join(path, "annotation", "top2_resetnet_feature.csv")), index=False)

In [4]:
top2_resnet.head(3)

,0,1
0,-1.245953,1.653044
1,-6.737771,3.163059
2,6.125465,-8.245798


The concatenation of spatial coordinates (image IDs) with the top 2 ResNet features to create a combined feature matrix

In [5]:
img_ids_list = np.load(os.path.join(path, "annotation", "img_ids_list.npy"))
n = np.zeros((len(img_ids_list), 2))
i = 0
j = 0

for vector in img_ids_list:
  x = int(vector.split('-')[0])
  y = int(vector.split('-')[1])
  n[i][0]=y
  n[i][1]=x
  i = i + 1

image_ids = pd.DataFrame(n)
resnet_feature_matrix_combined = pd.concat([image_ids, top2_resnet], axis=1, ignore_index=True)
resnet_feature_matrix_combined.rename(columns={0:'imagerow', 1:'imagecol', 2:'I1', 3:"I2"}, inplace=True )

In [6]:
resnet_feature_matrix_combined.head(3)

,imagerow,imagecol,I1,I2
0,6740.0,3781.0,-1.245953,1.653044
1,3259.0,2595.0,-6.737771,3.163059
2,7452.0,5843.0,6.125465,-8.245798


Subsequently, the corresponding barcode of each spot will be added to the previously generated matrix.

In [7]:
tissue_position = pd.read_csv((os.path.join(path, "spatial", "tissue_positions_list.csv")), sep=',', names=["barcode", "tissue", "row", "col","imagerow", "imagecol"])
combined_feature_matirx = pd.merge(tissue_position, resnet_feature_matrix_combined, how = "inner", on = ['imagerow', 'imagecol'])
adj_feature = combined_feature_matirx.drop(["tissue", "row", "col"], axis=1)
adj_feature.to_csv(os.path.join(path, "annotation", "adj_feature_matrix.csv"), index=False)

adj_feature.head(5)

,barcode,imagerow,imagecol,I1,I2
0,TAAGTAAATGTGCCGC-1,1610,6713,40.765457,23.894720
1,TGGCACGAGCTCGAGT-1,1609,6831,-3.993147,5.881485
2,ACCGGTCTGAGTACGG-1,1507,6889,158.223419,61.760967
3,GAACTTAGCGCCCGGT-1,1609,6948,-2.171487,-5.103039
4,AGTAGCTAGACGCCGA-1,1507,7007,18.478373,11.932409


The spatial coordinates were normalized using min-max normalization.

In [8]:
adj_feature_scaled = adj_feature.copy()

adj_feature_scaled["imagerow"] = (adj_feature_scaled["imagerow"] - 
                            adj_feature_scaled["imagerow"].min()) / (adj_feature_scaled["imagerow"].max() - adj_feature_scaled["imagerow"].min())    
adj_feature_scaled["imagecol"] = (adj_feature_scaled["imagecol"] - 
                            adj_feature_scaled["imagecol"].min()) / (adj_feature_scaled["imagecol"].max() - adj_feature_scaled["imagecol"].min())    

# view normalized data
display(adj_feature_scaled)

,barcode,imagerow,imagecol,I1,I2
0,TAAGTAAATGTGCCGC-1,0.014214,0.762694,40.765457,23.894720
1,TGGCACGAGCTCGAGT-1,0.014079,0.778461,-3.993147,5.881485
2,ACCGGTCTGAGTACGG-1,0.000271,0.786211,158.223419,61.760967
3,GAACTTAGCGCCCGGT-1,0.014079,0.794094,-2.171487,-5.103039
4,AGTAGCTAGACGCCGA-1,0.000271,0.801978,18.478373,11.932409
...,...,...,...,...,...
3589,ACAAACTCCATCAGAG-1,1.000000,0.316542,23.338575,12.756892
3590,TTGCGGAAAGCTGCCC-1,1.000000,0.332309,-4.329419,6.258057
3591,GGTTACCGCTCCCTAC-1,1.000000,0.347942,7.905270,19.481718
3592,CCGTTTCCTTTCCGTG-1,0.999865,0.363709,43.023933,41.408123


The adjacency matrix will be constructed using the cosine distance between the combined features of each node (spot). It is necessary to set the diagonal of the adjacency matrix to zero.

In [9]:
from sklearn.metrics import pairwise_distances

md = 1 - pairwise_distances(adj_feature_scaled.iloc[:,1:], metric="cosine")
md[md<0] = 0

#set the diagonal of the adjacency matrix to zero
adj = md - np.eye(len(md))
np.savetxt((os.path.join(path, "annotation", "adj.csv")), adj, delimiter=',')

In [10]:
adj = pd.DataFrame(adj)
adj.head(5)

,0,1,2,3,4,5,6,7,8,9,...,3584,3585,3586,3587,3588,3589,3590,3591,3592,3593
0,0.000000,0.0,0.987476,0.0,0.998861,0.000000,0.967066,0.000000,0.000000,0.995778,...,0.476086,0.539576,0.445267,0.988246,0.999234,0.998848,0.000000,0.792135,0.972083,0.974598
1,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.191477,0.850436,0.725241,0.000000,...,0.000000,0.000000,0.697695,0.000000,0.000000,0.000000,0.989587,0.553314,0.168665,0.157916
2,0.987476,0.0,0.000000,0.0,0.979329,0.000000,0.914814,0.000000,0.000000,0.997616,...,0.608448,0.664024,0.325930,0.999894,0.981251,0.991083,0.000000,0.686401,0.923210,0.927364
3,0.000000,0.0,0.000000,0.0,0.000000,0.971088,0.000000,0.000000,0.197545,0.000000,...,0.141506,0.071140,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.998861,0.0,0.979329,0.0,0.000000,0.000000,0.977166,0.000000,0.000000,0.990850,...,0.437622,0.503522,0.479846,0.980366,0.999363,0.996323,0.000000,0.817561,0.980871,0.982900


# Feature Matirx

In [24]:
#Read original 10x_h5 data and save it to h5ad
import scanpy as sc
from scanpy import read_10x_h5
adata = read_10x_h5(os.path.join(path, "filtered_feature_bc_matrix.h5"))
adata.var_names_make_unique()
spatial=pd.read_csv(os.path.join(path, "spatial", "tissue_positions_list.csv"), sep=",", header=None, na_filter=False,index_col=0) 
adata.obs["x1"]=spatial[1]

#Select captured samples
adata=adata[adata.obs["x1"]==1]
adata.var_names=[i.upper() for i in list(adata.var_names)]
adata.var["genename"]=adata.var.index.astype("str")
adata.write_h5ad(os.path.join(path, "FFD1.h5ad"))

#Read in gene expression and spatial location
adata=sc.read(os.path.join(path, "FFD1.h5ad"))

/Users/ninasong/opt/anaconda3/envs/gnn_git/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


The gene expression matrix for each barcode is contained within adata.X. To prepare this data for node feature generation, normalization and log-transformation will be performed.

In [25]:
adata.X.shape

(3594, 33538)

In [26]:
# Perform normalization
sc.pp.normalize_total(adata, target_sum=1e4)
# Log-transform the data
sc.pp.log1p(adata)

In [29]:
df.index

Index(['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', 'AAACAGAGCGACTCCT-1',
       'AAACAGCTTTCAGAAG-1', 'AAACAGGGTCTATATT-1', 'AAACAGTGTTCCTGGG-1',
       'AAACATTTCCCGGATT-1', 'AAACCCGAACGAAATC-1', 'AAACCGGGTAGGTACC-1',
       'AAACCGTTCGTCCAGG-1',
       ...
       'TTGTGTATGCCACCAA-1', 'TTGTGTTTCCCGAAAG-1', 'TTGTTAGCAAATTCGA-1',
       'TTGTTCAGTGTGCTAC-1', 'TTGTTCTAGATACGCT-1', 'TTGTTGTGTGTCAAGA-1',
       'TTGTTTCACATCCAGG-1', 'TTGTTTCATTAGTCTA-1', 'TTGTTTCCATACAACT-1',
       'TTGTTTGTATTACACG-1'],
      dtype='object', length=3594)

For gene expression data, we will take the top 10 principal components.

In [41]:
df = adata.to_df()
df.head(5)

pca = PCA(n_components=10)
embed = pca.fit_transform(df)

exp_emb = pd.DataFrame(embed)
exp_emb['barcode'] = df.index
barcode = exp_emb["barcode"]
exp_emb = exp_emb.drop(columns=['barcode'])
exp_emb.insert(loc=0, column='barcode', value=barcode)

The feature matrix was composed of the top 2 principal components of image features and the top 10 principal components of the gene expression matrix. These two matrices were meticulously aligned using the respective barcodes before being combined.

In [42]:
node_feature_matirx = pd.merge(adj_feature, exp_emb, how = "inner", on = ['barcode']).drop(columns=['imagerow', 'imagecol'])
node_feature_matirx.head(5)

,barcode,I1,I2,0,1,2,3,4,5,6,7,8,9
0,TAAGTAAATGTGCCGC-1,40.765457,23.894720,13.482821,3.819473,-2.406713,2.461850,2.822149,0.766574,1.429338,0.685947,-1.462653,-3.310129
1,TGGCACGAGCTCGAGT-1,-3.993147,5.881485,13.338982,2.251554,0.573883,0.856638,-0.965038,-0.146339,-0.396681,-1.578055,-0.792728,-3.522291
2,ACCGGTCTGAGTACGG-1,158.223419,61.760967,14.810644,1.708718,2.021286,-0.785960,0.158550,1.003724,1.736525,-0.027423,-1.515575,-2.939061
3,GAACTTAGCGCCCGGT-1,-2.171487,-5.103039,4.813560,0.737605,-3.233529,0.003260,-4.509124,0.538836,1.147582,-2.149444,-0.344673,-0.073488
4,AGTAGCTAGACGCCGA-1,18.478373,11.932409,12.216682,4.356026,1.892780,-0.723544,-2.442569,1.355514,-0.107893,-0.020426,-0.330318,-0.869030


In [43]:
node_feature = node_feature_matirx.iloc[:,1:]
np.savetxt((os.path.join(path, "annotation", "node_feature.csv")), node_feature, delimiter=',')

# Create MATLAB data files

In [46]:
import pandas as pd 
import numpy as np
import scipy.sparse as sp
import scipy
import scipy.io as sio
import scipy.sparse as sp

def read_dataset(data):
    #data = sio.loadmat(os.path.join('data', f'{dataset}.mat'))
    features = data['fea'].astype(float)
    adj = data['W']
    adj = adj.astype(float)
    if not sp.issparse(adj):
        adj = sp.csc_matrix(adj)
    if sp.issparse(features):
        features = features.toarray()
    labels = data['gnd'].reshape(-1) - 1
    n_classes = len(np.unique(labels))
    return adj, features, labels, n_classes

Although the current unsupervised algorithm is not designed to optimize for accuracy, it still calculates accuracy as a reference metric. To avoid potential errors, we will generate random labels for the evaluation process.

In [50]:
path = "/Users/ninasong/Desktop/spatialProject/literature_model/graph_convolutional_clustering/unsupervised-GCN/FFD1/"

#node = np.genfromtxt((os.path.join(path, "annotation", "node_feature.csv")), delimiter=',')
#adj = np.genfromtxt((os.path.join(path, "annotation", "adj.csv")), delimiter=',')

node = node_feature
adj = adj

# generate random labels
na_array = np.random.randint(6, size = node.shape[0])

#print("Node features:\n", node_feats)
print(node.shape[1])
print(adj.shape)

12
(3594, 3594)


csr_matrix is a compressed sparse row matrix format in Python's scipy.sparse module. It is a data structure is required in this tutorial, that is used to represent large, sparse matrices efficiently, where most of the elements are zero.

In a csr_matrix, the non-zero values are stored in three arrays:

1. The data array, which stores the non-zero values of the matrix in row-major order.
2. The indices array, which stores the column index of each non-zero value in the data array.
3. The indptr array, which stores the starting and ending indices of the non-zero values for each row in the data array.

Using this format, a sparse matrix can be represented efficiently in memory, and matrix operations can be performed efficiently without needing to store all the zeros in the matrix.

In [51]:
from scipy.sparse import csr_matrix

adj_sparse_matrix = csr_matrix(adj)
node_sparse_matrix = csr_matrix(node)
na_array = np.random.randint(low = 1, high = 11, size=node.shape[0])

In [52]:
# Define the file name for your .mat file
filename = 'FFD1.mat'

# Save the arrays to the .mat file
scipy.io.savemat(filename, {'W': adj_sparse_matrix, 'fea': node_sparse_matrix, 'gnd': na_array})

In [53]:
mat = scipy.io.loadmat(filename)
mat

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue Mar 14 23:34:30 2023',
 '__version__': '1.0',
 '__globals__': [],
 'W': <3594x3594 sparse matrix of type '<class 'numpy.float64'>'
 	with 7535277 stored elements in Compressed Sparse Column format>,
 'fea': <3594x12 sparse matrix of type '<class 'numpy.float64'>'
 	with 43128 stored elements in Compressed Sparse Column format>,
 'gnd': array([[ 8,  4, 10, ...,  6,  2,  5]])}